In [1]:
import pandas as pd
import sqlite3

In [2]:
data = pd.read_csv('.\Project\data.csv')

### Clean the data:
### 1.NaN in category_code
### 2.Inlucde sessions of user_id with at least 1 historical purchase
### 3.Extract only the first component of category_code

In [3]:
data.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
0,2019-10-01 00:00:00 UTC,view,44600062,2103807459595387724,NaN,shiseido,35.79,541312140,72d76fde-8bb3-4e00-8c23-a032dfed738c
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d


In [4]:
df = data[data['category_code'].notna()]
df.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
1,2019-10-01 00:00:00 UTC,view,3900821,2053013552326770905,appliances.environment.water_heater,aqua,33.20,554748717,9333dfbd-b87a-4708-9857-6336556b0fcc
2,2019-10-01 00:00:01 UTC,view,17200506,2053013559792632471,furniture.living_room.sofa,NaN,543.10,519107250,566511c2-e2e3-422b-b695-cf8e6e792ca8
3,2019-10-01 00:00:01 UTC,view,1307067,2053013558920217191,computers.notebook,lenovo,251.74,550050854,7c90fc70-0e80-4590-96f3-13c02c18c713
4,2019-10-01 00:00:04 UTC,view,1004237,2053013555631882655,electronics.smartphone,apple,1081.98,535871217,c6bd7419-2748-4c56-95b4-8cec9ff8b80d
5,2019-10-01 00:00:05 UTC,view,1480613,2053013561092866779,computers.desktop,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9


In [6]:
df.shape

(28933155, 9)

In [17]:
##customers who made at least one purchase:
purchasers = df.loc[df['event_type'] == 'purchase','user_id']
ps = set(purchasers)

In [19]:
pdf = df.loc[df['user_id'].isin(ps)]

In [30]:
pdf['category_code']=pdf['category_code'].str.split('.',expand= True).iloc[:,0]

<ipython-input-30-69da45dc12b0>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pdf['category_code']=pdf['category_code'].str.split('.',expand= True).iloc[:,0]


In [31]:
pdf.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
5,2019-10-01 00:00:05 UTC,view,1480613,2053013561092866779,computers,pulser,908.62,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
22,2019-10-01 00:00:22 UTC,view,1480714,2053013561092866779,computers,pulser,921.49,512742880,0d0d91c2-c9c2-4e81-90a5-86594dec0db9
23,2019-10-01 00:00:23 UTC,view,1004739,2053013555631882655,electronics,xiaomi,197.55,519530528,9882d21f-2c5f-496b-90d4-a1503edb6562
45,2019-10-01 00:00:35 UTC,view,3601505,2053013563810775923,appliances,samsung,463.15,526631741,5ec9bd77-beef-443e-b987-62ffb55b8132
47,2019-10-01 00:00:36 UTC,view,3600575,2053013563810775923,appliances,hotpoint-ariston,275.37,554754045,bd0302ef-c5ca-4b6a-b916-95cc2840c72c


### Group the data:
### Groupping and Counting the sessions by category and user_id
### Pivoting the result

In [37]:
grouped_df = pdf.loc[:,['event_type','category_code','user_id']].groupby(by = ['user_id','category_code'],as_index = False).count()

In [38]:
grouped_df.head()

,user_id,category_code,event_type
0,303160429,electronics,3
1,340041246,computers,8
2,340041246,electronics,21
3,384989212,appliances,106
4,384989212,computers,3


In [53]:
final_df = grouped_df.pivot(index = 'user_id',columns = 'category_code', values = 'event_type').reset_index().rename_axis(None,axis= 1).fillna(0).astype(int)

In [54]:
final_df2 = final_df.loc[:,final_df.columns != 'user_id'].add_prefix('count_of_sessions_')

In [55]:
final_df = pd.concat([final_df.loc[:,final_df.columns == 'user_id'],final_df2],axis = 1)
final_df.head()


,user_id,count_of_sessions_accessories,count_of_sessions_apparel,count_of_sessions_appliances,count_of_sessions_auto,count_of_sessions_computers,count_of_sessions_construction,count_of_sessions_country_yard,count_of_sessions_electronics,count_of_sessions_furniture,count_of_sessions_kids,count_of_sessions_medicine,count_of_sessions_sport,count_of_sessions_stationery
0,303160429,0,0,0,0,0,0,0,3,0,0,0,0,0
1,340041246,0,0,0,0,8,0,0,21,0,0,0,0,0
2,384989212,0,0,106,0,3,2,0,21,0,0,0,0,0
3,401021311,0,0,0,0,0,0,0,4,0,0,0,0,0
4,403013066,0,0,0,0,0,0,0,13,0,0,0,0,0


In [58]:
final_df.to_csv('.\Project\Wei.csv')

In [10]:
conn = sqlite3.connect('db')
c = conn.cursor()

c.execute('''
                create TABLE sessions (
                event_time TEXT
                event_type TEXT
                product_id INTEGER
                category_id INTEGER
                category_code TEXT
                brand TEXT
                price REAL
                user_id INTEGER
                user_session TEXT
                )
                ''')
conn.commit()